In [24]:
import pandas as pd
import numpy as np

In [46]:
df_csv = pd.read_csv('customer_signups.csv')
print("The data Info")
print(df_csv.info())


The data Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customer_id       298 non-null    object
 1   name              291 non-null    object
 2   email             266 non-null    object
 3   signup_date       298 non-null    object
 4   source            291 non-null    object
 5   region            270 non-null    object
 6   plan_selected     292 non-null    object
 7   marketing_opt_in  290 non-null    object
 8   age               288 non-null    object
 9   gender            292 non-null    object
dtypes: object(10)
memory usage: 23.6+ KB
None


In [ ]:

missing_count = df_csv.isnull().sum()
print("The nº of missing values per column")
print(missing_count)
missing_pct = (df_csv.isnull().mean() * 100).round(2)
print("The % of missing values per colum")
print(missing_pct)
missing_total = missing_pct.sum()
print("The total % of missing values are:",missing_total)

The nº of missing values per colum
customer_id          2
name                 9
email               34
signup_date          2
source               9
region              30
plan_selected        8
marketing_opt_in    10
age                 12
gender               8
dtype: int64
The % of missing values per colum
customer_id          0.67
name                 3.00
email               11.33
signup_date          0.67
source               3.00
region              10.00
plan_selected        2.67
marketing_opt_in     3.33
age                  4.00
gender               2.67
dtype: float64
The total % of missing values are: 41.34


In [37]:
df_csv['signup_date'] = pd.to_datetime(df_csv['signup_date'], errors='coerce' )
print(df_csv.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   customer_id       298 non-null    object        
 1   name              291 non-null    object        
 2   email             266 non-null    object        
 3   signup_date       294 non-null    datetime64[ns]
 4   source            291 non-null    object        
 5   region            270 non-null    object        
 6   plan_selected     292 non-null    object        
 7   marketing_opt_in  290 non-null    object        
 8   age               288 non-null    object        
 9   gender            292 non-null    object        
dtypes: datetime64[ns](1), object(9)
memory usage: 23.6+ KB
None


C:\Users\selma\AppData\Local\Temp\ipykernel_11820\2395876625.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_csv['signup_date'] = pd.to_datetime(df_csv['signup_date'], errors='coerce' )


In [45]:
duplicate_count = df_csv.duplicated(subset='customer_id').sum()
print("Nº of duplicate records are:", duplicate_count)
df_2 = df_csv.drop_duplicates(subset='customer_id', keep='first')
print("Nº of records whit duplicate", len(df_csv))
print("Nº of records whit no duplicate", len(df_2))

Nº of duplicate records are: 1
Nº of records whit duplicate 300
Nº of records whit no duplicate 299


In [69]:
df_2['plan_selected'].value_counts(dropna=False)

plan_selected
Premium        57
Pro            53
basic          46
Basic          46
PREMIUM        42
PRO            41
NaN             8
UnknownPlan     6
Name: count, dtype: int64

In [135]:
def standardize_plan(x): 
    if pd.isna(x): 
        return np.nan 
    
    s = str(x).strip().lower() 

    mapping = { 
        'premium': 'Premium', 
        'pro': 'Pro', 
        'basic': 'Basic',
        'unknownplan': 'Unknownplan' 
    } 

    for k, v in mapping.items():
        if s == k:            
            return v

    if s in {'basic', 'pro', 'premium','unknownplan'}:
        return s.title()

    return s.title()

df_2['plan_selected_std'] = df_2['plan_selected'].apply(standardize_plan)
print(df_2['plan_selected_std'].unique())
print(df_2['plan_selected_std'].value_counts(dropna=False))


['Basic' 'Premium' 'Pro' 'Unknownplan' nan]
plan_selected_std
Premium        99
Pro            94
Basic          92
NaN             8
Unknownplan     6
Name: count, dtype: int64


In [62]:
df_2['gender'].value_counts(dropna=False)

gender
Other         59
FEMALE        51
male          48
Male          44
Non-Binary    42
Female        41
NaN            8
123            6
Name: count, dtype: int64

In [133]:
def standardize_gender(x):
    if pd.isna(x):
        return np.nan
    
    s = str(x).strip().lower()

    mapping = {
        'male': 'Male',
        'female': 'Female',
        'other': 'Other',
        'non-binary': 'Non-Binary',
    }

    if s in mapping:
        return mapping[s]
        if s.isnumeric():  
            return 'Other'
    return 'Other'

    if s in {'male','female','non-binary','other'}:
        return s.title()
     
    return s.title()


df_2['gender_std'] = df_2['gender'].apply(standardize_gender)

print(df_2['gender_std'].value_counts(dropna=False))

gender_std
Female        92
Male          92
Other         65
Non-Binary    42
NaN            8
Name: count, dtype: int64


In [63]:
df_2['marketing_opt_in'].value_counts(dropna=False)

marketing_opt_in
No     156
Yes    132
NaN     10
Nil      1
Name: count, dtype: int64

In [84]:
df_2['marketing_opt_in'] = (
    df_2['marketing_opt_in']
    .replace({'Nil': np.nan, 'nil': np.nan})
)
# The yes and no are alrredy standardise in text values
df_2['marketing_opt_in'].value_counts(dropna=False)

marketing_opt_in
No     156
Yes    132
NaN     11
Name: count, dtype: int64

In [136]:

df_2['signup_date'] = pd.to_datetime(df_2['signup_date'], errors='coerce')
signups_per_week = (
    df_2
    .dropna(subset=['signup_date'])
    .assign(week=lambda x: x['signup_date'].dt.to_period('W').apply(lambda r: r.start_time))
    .groupby('week')
    .size()
    .rename('signups')
    .reset_index()
    .sort_values('week')
)

print(signups_per_week)

         week  signups
0  2024-01-01        6
1  2024-01-08        5
2  2024-01-15        7
3  2024-01-22        7
4  2024-01-29        8
5  2024-02-05        6
6  2024-02-12        6
7  2024-02-19        7
8  2024-02-26        7
9  2024-03-04        7
10 2024-03-11        5
11 2024-03-18        6
12 2024-03-25        6
13 2024-04-01        7
14 2024-04-08        5
15 2024-04-15        7
16 2024-04-22        7
17 2024-04-29        6
18 2024-05-06        4
19 2024-05-13        7
20 2024-05-20        7
21 2024-05-27        7
22 2024-06-03        7
23 2024-06-10        5
24 2024-06-17        7
25 2024-06-24        7
26 2024-07-01        7
27 2024-07-08        5
28 2024-07-15        6
29 2024-07-22        7
30 2024-07-29        7
31 2024-08-05        5
32 2024-08-12        6
33 2024-08-19        7
34 2024-08-26        7
35 2024-09-02        7
36 2024-09-09        5
37 2024-09-16        7
38 2024-09-23        7
39 2024-09-30        7
40 2024-10-07        5
41 2024-10-14        6
42 2024-10-

In [91]:
signups_by_source = df_2['source'].value_counts(dropna=False).rename_axis('source').reset_index(name='signups')
signups_by_region = df_2['region'].value_counts(dropna=False).rename_axis('region').reset_index(name='signups')
signups_by_plan = df_2['plan_selected_std'].value_counts(dropna=False).rename_axis('plan_selected').reset_index(name='signups')

In [ ]:
signups_by_source = (
    df_2['source']
    .value_counts(dropna=False)
    .rename_axis('source')
    .reset_index(name='signups')
)

signups_by_region = (
    df_2['region']
    .value_counts(dropna=False)
    .rename_axis('region')
    .reset_index(name='signups')
)

signups_by_plan = (
    df_2['plan_selected_std']
    .value_counts(dropna=False)
    .rename_axis('plan_selected')
    .reset_index(name='signups')
)



      source  signups
0    YouTube       58
1     Google       50
2   Referral       49
3  Instagram       48
4   Facebook       40
    region  signups
0    North       65
1     East       61
2    South       59
3     West       45
4  Central       39
  plan_selected  signups
0       Premium       99
1           Pro       94
2         Basic       92
3           NaN        8
4   Unknownplan        6


In [106]:
df_2['age'] = pd.to_numeric(df_2['age'], errors='coerce') 
df_2['age'].describe()


count    280.00000
mean      36.17500
std       14.99802
min       21.00000
25%       25.00000
50%       34.00000
75%       40.00000
max      206.00000
Name: age, dtype: float64

In [107]:
age_series = df_2['age'].dropna()

age_summary = pd.DataFrame({
    'min': [age_series.min()],
    'max': [age_series.max()],
    'mean': [age_series.mean()],
    'median': [age_series.median()],
    'null_count': [df_2['age'].isna().sum()]
})

age_summary

,min,max,mean,median,null_count
0,21.0,206.0,36.175,34.0,19


In [ ]:
### Answer These Business QuesAons

In [ ]:
### 1. Which acquisi4on source brought in the most users last month?

last_month = df_2['signup_date'].dt.to_period('M').max()
mask = df_2['signup_date'].dt.to_period('M') == last_month
last_month_data = df_2[mask]
top_source = last_month_data['source'].value_counts(dropna=False).idxmax()
top_source_count = last_month_data['source'].value_counts(dropna=False).max()

print("Top source last month:", top_source, top_source_count)

Top source last month: Instagram 4


In [ ]:
###2) Which region shows signs of missing or incomplete data?
key_cols = ['email','age','gender_std','plan_selected_std','marketing_opt_in_std']


use_cols = [c for c in key_cols if c in df_2.columns]
if not use_cols:
    raise ValueError(f"Nenhuma coluna disponível entre {key_cols}. Crie-as antes (ex.: *_std).")

df_2 = df_2.copy()
df_2['region_fixed'] = df_2['region'].fillna("Missing Region")

region_missing = (
    df_2
    .groupby('region_fixed')[use_cols]
    .apply(lambda g: g.isna().mean())   # % missing por coluna
    .mean(axis=1)                       # média por região
    .sort_values(ascending=False)
)

region_most_incomplete = region_missing.index[0]
region_most_incomplete_rate = float(region_missing.iloc[0])

print("Region with most incomplete data:", region_most_incomplete, f"{region_most_incomplete_rate:.3%}")

Region with most incomplete data: East 6.885%


In [116]:
###3) Are older users more or less likely to opt in to marketing?
df_2['optin_binary'] = df_2['marketing_opt_in_std'].map({'Yes':1,'No':0})

corr = df_2[['age','optin_binary']].corr(method='spearman').iloc[0,1]
print("Spearman correlation (age vs opt-in):", corr)

if corr > 0:
    print("→ Older users are slightly MORE likely to opt in.")
elif corr < 0:
    print("→ Older users are slightly LESS likely to opt in.")
else:
    print("→ No relationship.")


Spearman correlation (age vs opt-in): 0.03703325788763671
→ Older users are slightly MORE likely to opt in.


In [ ]:
###4) Which plan is most commonly selected, and by which age group?

bins = [0, 24, 34, 44, 54, 200]
labels = ['<25','25–34','35–44','45–54','55+']
df_2['age_group'] = pd.cut(df_2['age'], bins=bins, labels=labels, right=True)


top_plan = df_2['plan_selected_std'].value_counts().idxmax()


top_age_group = (
    df_2[df_2['plan_selected_std'] == top_plan]['age_group']
    .value_counts()
    .idxmax()
)

print("Most common plan:", top_plan)
print("Dominant age group for this plan:", top_age_group)


Most common plan: Premium
Dominant age group for this plan: 25–34


In [128]:
###5) (Optional) Which plan’s users are most likely to contact support?
support = pd.read_csv("support_tickets.csv")
merged = df_2.merge(support, on='customer_id', how='left')
support_by_plan = (
    merged
    .groupby('plan_selected_std')['ticket_id']
    .count()
    .sort_values(ascending=False)
)

print(support_by_plan)


plan_selected_std
Pro            47
Basic          42
Premium        26
Unknownplan     4
Name: ticket_id, dtype: int64


In [ ]:
support['ticket_date'] = pd.to_datetime(support['ticket_date'], errors='coerce')
merged1 = support.merge(
    df_2[['customer_id', 'signup_date']], 
    on='customer_id', 
    how='left'
)

merged1['days_diff'] = (merged1['ticket_date'] - merged1['signup_date']).dt.days
within_2w = merged1[(merged1['days_diff'] >= 0) & (merged1['days_diff'] <= 14)]
customers_within_2w = within_2w['customer_id'].nunique()
print("Customers contacting support within 2 weeks:", customers_within_2w)


Customers contacting support within 2 weeks: 47
